Here I just want to test out my various algorithms to make sure they're actually working properly. 

In [1]:
from funcs import *
from multi_funcs import *

In [48]:
df = mdf_np(10,5)

In [49]:
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,5,7,9,6,8,0,0,0,False,0.967690,0.125583,0.552282,0.930853,0.165301,True
1,6,8,0,9,1,1,0,0,False,0.380894,0.452325,0.205760,0.881078,0.344232,True
2,9,3,0,7,6,2,0,0,False,0.843450,0.105715,0.685080,0.844282,0.719757,True
3,3,0,5,9,7,3,0,0,False,0.161275,0.319044,0.160289,0.997159,0.414404,True
4,0,5,3,6,9,4,0,0,False,0.819647,0.128617,0.854167,0.756092,0.820660,True
5,8,1,5,3,7,5,0,0,False,0.030736,0.803023,0.657167,0.571345,0.002931,True
6,2,9,4,8,3,6,0,0,False,0.156353,0.429465,0.016706,0.504242,0.698684,True
7,3,1,8,7,9,7,0,0,False,0.552368,0.816995,0.052622,0.512400,0.583048,True
8,1,4,0,6,5,8,0,0,False,0.298489,0.325296,0.390549,0.298158,0.020683,True
9,5,2,0,6,7,9,0,0,False,0.917484,0.116078,0.070496,0.829199,0.643030,True


In [50]:
df[0] = np.arange(10)

In [51]:
df = df.iloc[:6,:]

In [52]:
df[0] = [0,1,2,3,4,4]
df['rank1'] = [0,0,0,0,0,1]
df['rank2'] = [1.1, 1.1, 1.1, 1.1, 1.1, 0]
df[1] = [1,2,3,4,0,5]

In [53]:
df['k'] = 0
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,0,1,9,6,8,0,0,0,False,0,1.1,0.552282,0.930853,0.165301,True
1,1,2,0,9,1,1,0,0,False,0,1.1,0.205760,0.881078,0.344232,True
2,2,3,0,7,6,2,0,0,False,0,1.1,0.685080,0.844282,0.719757,True
3,3,4,5,9,7,3,0,0,False,0,1.1,0.160289,0.997159,0.414404,True
4,4,0,3,6,9,4,0,0,False,0,1.1,0.854167,0.756092,0.820660,True
5,4,5,5,3,7,5,0,0,False,1,0.0,0.657167,0.571345,0.002931,True


In [57]:
def run_gale_shapley(sp, k = 3):
    '''Runs the Gale-Shapley Algorithm for a particular dataframe.
    ______
    sp: dataframe of student preferences and school scores over those students.    
    '''
    r_apps = 0
    i=0
    mask = [1]
    while r_apps < k and sum(mask) > 0: ## note that there are two ways to end the cycle. 1 all unmatched students have applied to 3 schools. 2. There are no unmatched students.
        ## Step 1: Find k
        sp = find_k(sp)
        #sp['underdemanded'] = np.where(sp['k']>1, False, sp['underdemanded'])

        ## Step 2: Find matches
        matches = find_matches_3(sp) ## finds first round of matches.

        ## Step 3: Create mask and apply to matched column
        mask = create_mask(matches, sp, k) ## Creates a mask of students who have not been assigned to a school.
        sp.loc[:,'matched'] = mask

        ## Step 4: Update the underdemanded column
        condition = np.logical_or(sp['k'] > 1, sp['undermatched'] == False)
        sp['underdemanded'] = np.where(condition & sp['matched'] == True, False, True)
        sp['underdemanded'] = np.where(sp['matched'] == False, True, sp['underdemanded'])


        ## Step 5: Shift Values Over for unmatched
        sp = shift(sp, mask, k)

        ## Step 6: Update N, to reflect total number of matches for each school. 
        #sp['N'] += matches.set_index(0)['k'].reindex(sp[0], fill_value=0).reset_index(drop=True)

        ## Step 7: Remove all rejects who have applied to k schools:
        sp = sp[sp['applications'] < k] # If you've been rejected from three schools, you're out. We also don't want you to stay in the dataset. 
        sp.reset_index(drop=True, inplace=True) # Reset the index
        i+=1
    return sp, i

In [58]:
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,0,1,9,6,8,0,0,1,False,0,1.1,0.552282,0.930853,0.165301,True
1,1,2,0,9,1,1,0,1,False,0,1.1,0.205760,0.881078,0.344232,True
2,2,3,0,7,6,2,0,1,False,0,1.1,0.685080,0.844282,0.719757,True
3,3,4,5,9,7,3,0,1,False,0,1.1,0.160289,0.997159,0.414404,True
4,4,0,3,6,9,4,0,2,False,0,1.1,0.854167,0.756092,0.820660,True
5,4,5,5,3,7,5,0,2,False,1,0.0,0.657167,0.571345,0.002931,True


In [ ]:
sp = find_k(sp)
        sp['underdemanded'] = np.where(sp['k']>1, False, sp['underdemanded'])

        ## Step 2: Find matches
        matches = find_matches_3(sp) ## finds first round of matches.

        ## Step 3: Create mask and apply to matched column
        mask = create_mask(matches, sp, k) ## Creates a mask of students who have not been assigned to a school.
        sp.loc[:,'matched'] = mask
        
        ## Step 4: Shift Values Over for unmatched
        sp = shift(sp, mask, k)

        ## Step 5: Update N, to reflect total number of matches for each school. 
        #sp['N'] += matches.set_index(0)['k'].reindex(sp[0], fill_value=0).reset_index(drop=True)

        ## Step 6: Remove all rejects who have applied to k schools:
        sp = sp[sp['applications'] < k] # If you've been rejected from three schools, you're out. We also don't want you to stay in the dataset. 
        sp.reset_index(drop=True, inplace=True) # Reset the index

In [47]:
run_gale_shapley(df)

(   0  1  2  3  4  student_id  applications  k  matched  rank1     rank2  \
 0  1  0  0  8  5           0             1  1    False    1.1  0.564900   
 1  2  3  3  1  9           1             1  1    False    1.1  0.012981   
 2  3  2  2  0  5           2             1  1    False    1.1  0.612458   
 3  4  8  8  0  7           3             1  1    False    1.1  0.109342   
 4  0  9  9  4  6           4             1  1    False    1.1  0.754992   
 5  5  5  5  1  0           5             1  1    False    0.0  0.938565   
 
       rank3     rank4     rank5  underdemanded  
 0  0.564900  0.348627  0.147978          False  
 1  0.012981  0.661379  0.562983          False  
 2  0.612458  0.750695  0.861968          False  
 3  0.109342  0.792886  0.794326          False  
 4  0.754992  0.539835  0.498842          False  
 5  0.938565  0.127579  0.192676          False  ,
 7)

In [31]:
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,0,1,6,7,1,0,0,1,False,0.0,1.100000,0.567503,0.697280,0.079864,True
1,1,2,9,0,8,1,0,1,False,0.0,1.100000,0.371542,0.086808,0.831132,True
2,2,3,7,6,8,2,0,1,False,0.0,1.100000,0.782155,0.302918,0.729418,True
3,3,4,2,8,9,3,0,1,False,0.0,1.100000,0.040249,0.148500,0.607788,True
4,0,6,6,3,4,4,1,2,True,1.1,0.676477,0.676477,0.973664,0.195210,False
5,4,5,8,5,2,5,0,2,False,1.0,0.000000,0.231485,0.309091,0.950274,False
